In [1]:
%matplotlib inline
# A linha acima é necessária para visualizar os gráficos no jupyter notebook

import os
import pandas as pd
# Módulo com funções matemáticas para arrays multidimensionais (matrizes)
import numpy as np
# Módulo com funções de geração de gráficos
import matplotlib.pyplot as plt


In [28]:
# Carregando os dados
carne_produzida = pd.read_csv('meat-production-tonnes.csv')
pais_carne_produzida = carne_produzida['Entity']
ano_carne_produzida = carne_produzida['Year']
ano2010_carne_produzida = ano_carne_produzida == '2010' 


carne_produzida.head(5)

,Entity,Code,Year,"Livestock Primary - Meat, Total - 1765 - Production - 5510 - tonnes (tonnes)"
0,Afghanistan,AFG,1961,129420
1,Afghanistan,AFG,1962,132206
2,Afghanistan,AFG,1963,138971
3,Afghanistan,AFG,1964,143830
4,Afghanistan,AFG,1965,150195


In [30]:
carne_2010 = 
for i in ano2010_carne_produzida:
    if i == True:
        carne_2010[''
        

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49     True
Name: Year, dtype: bool

In [5]:
target_consumo_per_capita = pd.read_csv('per-capita-meat-consumption-by-type-kilograms-per-year.csv')
target_consumo_per_capita.head(5)

,Entity,Code,Year,Mutton and Goat (kilograms per year),Other meats (kilograms per person),Poultry (kilograms per year),Pigmeat (kilograms per year),Beef and bufallo (kilograms per year)
0,Afghanistan,AFG,1961,8.18,0.85,0.63,NaN,4.80
1,Afghanistan,AFG,1962,7.92,0.88,0.66,NaN,5.01
2,Afghanistan,AFG,1963,8.09,1.07,0.66,NaN,5.06
3,Afghanistan,AFG,1964,8.35,1.01,0.67,NaN,5.03
4,Afghanistan,AFG,1965,8.64,1.06,0.70,NaN,4.99


In [18]:
target_IDH = pd.read_csv('HDI.csv')
target_IDH.head(5)

,Unnamed: 0,Id,Country,HDI Rank,HDI,Life expectancy,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,Change in HDI rank 2010-2015,...,Coefficient of human inequality,Inequality in life expectancy (%) 2010-2015,Inequality-adjusted life expectancy index,Inequality in education(%),Inequality-adjusted education index,Inequality in income (%),Inequality-adjusted income index,Income inequality (Quintile ratio) 2010-2015,Income inequality (Palma ratio) 2010-2015,Income inequality (Gini coefficient) 2010-2015
0,0,1,Norway,1.0,0.949,81.7,12.7,67614.0,5.0,0.0,...,5.4,3.3,0.918,2.4,0.894,10.4,0.882,3.8,0.9,25.9
1,1,2,Australia,2.0,0.939,82.5,13.2,42822.0,19.0,1.0,...,8.0,4.3,0.921,1.9,0.921,17.7,0.753,6.0,1.4,34.9
2,2,3,Switzerland,2.0,0.939,83.1,13.4,56364.0,7.0,0.0,...,8.4,3.8,0.934,5.7,0.840,15.7,0.806,4.9,1.2,31.6
3,3,4,Germany,4.0,0.926,81.1,13.2,45000.0,13.0,0.0,...,7.0,3.7,0.905,2.6,0.891,14.8,0.787,4.6,1.1,30.1
4,4,5,Denmark,5.0,0.925,80.4,12.7,44519.0,13.0,2.0,...,7.0,3.8,0.894,3.0,0.896,14.3,0.789,4.5,1.0,29.1
